In [ ]:
#########################################################################
## COMPROBAR GPU ASIGNADA EN COLABORATORY
#########################################################################
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Mar  5 11:40:14 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| N/A   59C    P8    19W / 100W |     54MiB /  6144MiB |     41%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#########################################################################
## LIBRERIAS NECESARIAS
#########################################################################
import tensorflow as tf
import gymnasium as gym
import sinergym 
from sinergym.utils.wrappers import (LoggerWrapper, NormalizeAction,
                                     NormalizeObservation) 
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
import keras


# Librerias necesarias para BC
from stable_baselines3.common.evaluation import evaluate_policy

from imitation.algorithms import bc
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
# Problema en rollout: es la función que define las transiciones expertas
import imitation.data.rollout as rollout 
from stable_baselines3.common.vec_env import DummyVecEnv


2024-03-05 11:40:23.813289: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# **5Zone**

## 1. Creamos un vector de entornos

In [ ]:
def _make_env():
     _env = gym.make("Eplus-5zone-hot-discrete-v1")
     _env = NormalizeObservation(_env)
     _env = LoggerWrapper(_env)
     #_env = RolloutInfoWrapper(_env)
     return _env

# https://stable-baselines.readthedocs.io/en/master/guide/vec_envs.html
venv = DummyVecEnv([_make_env for _ in range(1)])

#==============================================================================================#
#==============================================================================================#


## 2.Creación de demostraciones expertas 

In [ ]:
env = gym.make("Eplus-5zone-hot-discrete-v1")
env = NormalizeObservation(env)
env = LoggerWrapper(env)

#==============================================================================================#
#==============================================================================================#


In [ ]:
# Función que devuekve la política experta 
def download_expert():
    print("Downloading a pretrained expert.")
    # https://imitation.readthedocs.io/en/latest/main-concepts/experts.html
    expert = load_policy(
        "ppo",
        path="model5zone.zip",
        venv=env,
    )
    return expert

# Función que devuelve trayectorias de la política experta 
def sample_expert_transitions():
    # Cargamos la política experta
    expert = download_expert()

    print("Sampling expert transitions.")

    # Generar trayectorias a partir de una política dada
    rollouts = rollout.rollout(
        expert,  # Política 
        venv,    # Entorno
        sample_until=rollout.make_sample_until(min_timesteps=None, min_episodes=1),  # EPISODES=1 asi que min_episodes=1
        rng=np.random.default_rng(),
        unwrap=False,
    )
    
    return rollout.flatten_trajectories(rollouts)


In [ ]:
# Selección de una muestra de trayectorias de secuencias expertas
transitions = sample_expert_transitions()

Sampling expert transitions.
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


In [ ]:
"""reward_mean, reward_std = evaluate_policy(agent.policy, venv, N_EVAL_EPISODES)
reward_stderr = reward_std/np.sqrt(N_EVAL_EPISODES)

print(f"Reward: {reward_mean:.0f} +/- {reward_stderr:.0f}")"""

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


...

Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#
Reward: -25131 +/- 5


## **Soft Q Imitation Learning (SQIL)**
* El aprendizaje de imitación Soft Q aprende a imitar una política a partir de demostraciones utilizando el algoritmo DQN con recompensas modificadas. 
* Durante cada actualización de la política, la mitad del lote se muestrea de las demostraciones y la otra mitad se muestrea del entorno. Las demostraciones de expertos se les asigna una recompensa de 1, y al entorno se le asigna una recompensa de 0. Esto anima a la política a imitar las demostraciones y, al mismo tiempo, a evitar estados no vistos en las demostraciones.

In [ ]:
from imitation.algorithms import sqil

### Experimento 1

In [ ]:
sqil_trainer = sqil.SQIL(
    venv=venv,
    demonstrations=transitions,
    policy="MlpPolicy",
)

In [ ]:
sqil_trainer.train(total_timesteps=3504)

In [ ]:
reward, _ = evaluate_policy(sqil_trainer.policy, sqil_trainer.venv, 5)
print("Reward:", reward)

### Experimento 2

In [ ]:
sqil_trainer = sqil.SQIL(
    venv=venv,
    demonstrations=transitions,
    policy="MlpPolicy",
)

In [ ]:
sqil_trainer.train(total_timesteps=7008)

In [ ]:
reward, _ = evaluate_policy(sqil_trainer.policy, sqil_trainer.venv, 5)
print("Reward:", reward)

### Experimento 3

In [ ]:
sqil_trainer = sqil.SQIL(
    venv=venv,
    demonstrations=transitions,
    policy="MlpPolicy",
)

In [ ]:
sqil_trainer.train(total_timesteps=10512)

In [ ]:
reward, _ = evaluate_policy(sqil_trainer.policy, sqil_trainer.venv, 5)
print("Reward:", reward)

### Experimento 4

In [ ]:
sqil_trainer = sqil.SQIL(
    venv=venv,
    demonstrations=transitions,
    policy="MlpPolicy",
)

In [ ]:
sqil_trainer.train(total_timesteps=14016)

In [ ]:
reward, _ = evaluate_policy(sqil_trainer.policy, sqil_trainer.venv, 5)
print("Reward:", reward)

### Experimento 5

In [ ]:
sqil_trainer = sqil.SQIL(
    venv=venv,
    demonstrations=transitions,
    policy="MlpPolicy",
)

In [ ]:
sqil_trainer.train(total_timesteps=17520)

In [ ]:
reward, _ = evaluate_policy(sqil_trainer.policy, sqil_trainer.venv, 5)
print("Reward:", reward)

### Experimento 6

In [ ]:
sqil_trainer = sqil.SQIL(
    venv=venv,
    demonstrations=transitions,
    policy="MlpPolicy",
)

In [ ]:
sqil_trainer.train(total_timesteps=21024)

In [ ]:
reward, _ = evaluate_policy(sqil_trainer.policy, sqil_trainer.venv, 5)
print("Reward:", reward)

### Experimento 7

In [ ]:
sqil_trainer = sqil.SQIL(
    venv=venv,
    demonstrations=transitions,
    policy="MlpPolicy",
)

In [ ]:
sqil_trainer.train(total_timesteps=24528)

In [ ]:
reward, _ = evaluate_policy(sqil_trainer.policy, sqil_trainer.venv, 5)
print("Reward:", reward)

### Experimento 8

In [ ]:
sqil_trainer = sqil.SQIL(
    venv=venv,
    demonstrations=transitions,
    policy="MlpPolicy",
)

In [ ]:
sqil_trainer.train(total_timesteps=28032)

In [ ]:
reward, _ = evaluate_policy(sqil_trainer.policy, sqil_trainer.venv, 5)
print("Reward:", reward)

### Experimento 9

In [ ]:
sqil_trainer = sqil.SQIL(
    venv=venv,
    demonstrations=transitions,
    policy="MlpPolicy",
)

In [ ]:
sqil_trainer.train(total_timesteps=31536)

In [ ]:
reward, _ = evaluate_policy(sqil_trainer.policy, sqil_trainer.venv, 5)
print("Reward:", reward)

### Experimento 10

In [ ]:
"""sqil_trainer = sqil.SQIL(
    venv=venv,
    demonstrations=transitions,
    policy="MlpPolicy",
)"""

In [ ]:
#sqil_trainer.train(total_timesteps=35038)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


In [ ]:
#reward, _ = evaluate_policy(sqil_trainer.policy, sqil_trainer.venv, 5)
#print("Reward:", reward)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


...

Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#


/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Progress: |****************************************************************************************************| 100%
#----------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------#
Reward: -23136.779058059212
